In [6]:
import csv
import pandas as pd
import numpy as np
import re
import os


'/Users/iago/code/CafedirectProducersFoundation/DataPrep'

In [7]:
kenya = pd.read_csv('KenyaClean.csv')
mapping_df = pd.read_csv('kenya_narrativedata_mapping.csv')

In [14]:
mapping_df[['Q21.Income.Other', ]

0                                                    NaN
1                                                  loans
2                                               pluckers
3                                          Farming costs
4                                              Cow feeds
5                                            development
6                                           workers wage
7                                                farming
8                                            development
9                                development buying land
10     invest on developments, pay wages,contribute t...
11                                                 wages
12                                    wages, development
13                                   wages, developments
14                                wages, improve my farm
15                             wages,saving,animal feeds
16                                                wages,
17                             

In [13]:
target_cols = ['Q21','Q93', 'Q28', 'Q95', 'Q97', 'Q99', 'Q147', 'Q148', 'Q150', 'Q162', 'Q163', 'Q165', 'Q166', 'Q170', 'Q171']

mapping_cols = [col for target in target_cols for col in mapping_df.columns if target in col and not '.FreeText' in col and not '.Clean' in col]
print(mapping_cols)

data_cols = [col for target in target_cols for col in kenya.columns if target in col and not '.FreeText' in col and not '.Clean' in col]
print(data_cols)

['Q21.Income.Other', 'Q21.Income.Other.Reponse1.Level1', 'Q21.Income.Other.Reponse1.Level2', 'Q21.Income.Other.Reponse2.Level1', 'Q21.Income.Other.Reponse2.Level2', 'Q93.BankAccountChallenges', 'Q28.FarmHealth.Text', 'Q95.BankLoanFacilityChallenges', 'Q95.BankLoanFacilityChallenges.Reponse1.Level1', 'Q95.BankLoanFacilityChallenges.Reponse1.Level2', 'Q95.BankLoanFacilityChallenges.Reponse2.Level1', 'Q97.BankCreditChallenges', 'Q99.SACCOChallenges', 'Q99.SACCOChallenges.Reponse1.Level1', 'Q99.SACCOChallenges.Reponse1.Level2', 'Q147.OtherInfoAreas', 'Q147.OtherInfoAreas.Reponse1.Level1', 'Q147.OtherInfoAreas.Reponse1.Level2', 'Q147.OtherInfoAreas.Reponse1.Level3', 'Q147.OtherInfoAreas.Reponse2.Level1', 'Q147.OtherInfoAreas.Reponse2.Level2', 'Q147.OtherInfoAreas.Reponse3.Level1', 'Q147.OtherInfoAreas.Reponse3.Level1.1', 'Q147.OtherInfoAreas.Reponse4.Level1', 'Q147.OtherInfoAreas.Reponse5.Level1', 'Q147.OtherInfoAreas.Reponse6.Level1', 'Q147.OtherInfoAreas.Reponse7.Level1', 'Q148.IdeasOfCha

In [21]:
sub_columns = mapping_df[['Q21.Income.Other',
 'Q21.Income.Other.Reponse1.Level1',
 'Q21.Income.Other.Reponse1.Level2',
 'Q21.Income.Other.Reponse2.Level1',
 'Q21.Income.Other.Reponse2.Level2']]

sub_columns = sub_columns.rename(columns={'Q21.Income.Other': 'Q21.Income.Other.O6.Text'})

sub_columns


,Q21.Income.Other.O6.Text,Q21.Income.Other.Reponse1.Level1,Q21.Income.Other.Reponse1.Level2,Q21.Income.Other.Reponse2.Level1,Q21.Income.Other.Reponse2.Level2
0,NaN,NaN,NaN,NaN,NaN
1,loans,Loans,NaN,NaN,NaN
2,pluckers,Wages,NaN,NaN,NaN
3,Farming costs,Farm Costs,NaN,NaN,NaN
4,Cow feeds,Livestock,NaN,NaN,NaN
5,development,Farm Costs,NaN,NaN,NaN
6,workers wage,Wages,NaN,NaN,NaN
7,farming,Farm Costs,NaN,NaN,NaN
8,development,Farm Costs,NaN,NaN,NaN
9,development buying land,Farm Costs,NaN,NaN,NaN


In [23]:
merged = pd.merge(kenya, sub_columns, how='inner', on=['Q21.Income.Other.O6.Text' ])

merged[['Q21.Income.Other.O6.Text','Q21.Income.Other.Reponse1.Level1',
 'Q21.Income.Other.Reponse1.Level2',
 'Q21.Income.Other.Reponse2.Level1',
 'Q21.Income.Other.Reponse2.Level2']]

,Q21.Income.Other.O6.Text,Q21.Income.Other.Reponse1.Level1,Q21.Income.Other.Reponse1.Level2,Q21.Income.Other.Reponse2.Level1,Q21.Income.Other.Reponse2.Level2
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN
